# Un peu d'Exploratory Data Analysis avant toutes choses

Ce notebook vise à explorer rapidement la base de données pour avoir une idée de se composition, la répartition et éventuels déséquilibres entre les classes, les caractéristiques des textes à analyser. Cela est éventuellement l'occasion de détecter des erreurs de textes (encodage erroné au moment d'enregistrer/lire les demandes)

# 1. Packages usuels et importation de la base train

In [1]:
# Pour faciliter la mise à jour des fonctions écrites dans func_custom sans avoir à redémarrer le kernel
%load_ext autoreload
%autoreload 2

In [2]:
# Packages classiques
import pandas as pd

# Custom package
import func_custom as fc

# NLP
from wordcloud import WordCloud
from nltk.corpus import stopwords

# Dataviz
import seaborn as sns
import matplotlib.pyplot as plt

La variable `Unnamed: 0` semble juste être un indice de ligne, on peut pour le moment ne pas le garder pour l'analyse

In [3]:
df_train = pd.read_excel("data/data_train.xlsx",
                         usecols = ["label", "message"],
                         index_col = False)

# 2. Preprocessing

In [4]:
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import SnowballStemmer

# Téléchargement des données nécessaires pour NLTK
nltk.download('punkt')
nltk.download('stopwords')

# Texte d'exemple
texte = "Voici un exemple de texte en français à prétraiter avec NLTK."

# Tokenisation
tokens = word_tokenize(texte, language='french')

# Suppression de la ponctuation et des stop words
stop_words = set(stopwords.words('french'))
tokens = [mot for mot in tokens if mot.isalnum() and mot not in stop_words]

# Lemmatisation ou racinisation
stemmer = SnowballStemmer('french')
tokens = [stemmer.stem(mot) for mot in tokens]

print(tokens)

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\TitouanBlaize\AppData\Roaming\nltk_data...


['voic', 'exempl', 'text', 'franc', 'prétrait', 'nltk']


[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\TitouanBlaize\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
